In [1]:
import pandas as pd
import numpy as np
import os
import glob
import ast
import sys

from collections import Counter

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import neural_network_functions as neural_net_fun, manual_labeler_functions as man_lab_fun, general_functions as gf, files_paths as fp

In [2]:
MAIN_LIST_LABELER = gf.find_files_in_all_subdirectories([fp.DATASET_SEED], fp.VD_LABELED_L0)

In [3]:
MAIN_LIST_LABELER

['..\\Dataset\\REF-Gold-Label\\VD_R_0000000001\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000003\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000005\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000007\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000008\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000009\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000010\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000011\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000012\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000013\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000014\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000015\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000016\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000018\\VD_LABELED_L0.CSV',
 '..\\Dataset\\REF-Gold-Label\\VD_R_0000000019\\

In [4]:
num_file_sequences = len(gf.find_files_in_all_subdirectories([os.path.join('.', 'result_sequences')], '*.CSV'))

In [5]:
gf.find_files_in_all_subdirectories([os.path.join('.', 'result_sequences')], '*.csv')

['.\\result_sequences\\DATASET_SEQUENCES_1.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_2.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_3.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_4.CSV']

In [6]:
num_file_sequences

4

In [7]:
OUTPUT_NAME = os.path.join('result_sequences','DATASET_SEQUENCES_' + str(num_file_sequences + 1) + '.CSV')
DATASET_SEQUENCES = pd.DataFrame()
idx_sample = 0
for current_path_location in MAIN_LIST_LABELER:
    VD_LABELED_DT = pd.read_csv(current_path_location)
    VD_LABELED_DT.drop(columns=['Unnamed: 0'], inplace=True)
    labels = man_lab_fun.GET_ALL_CLASSES(VD_LABELED_DT)
    print("labels:", labels)
    print(current_path_location)
    final_df = pd.DataFrame()
    for label in labels:
        
        # Usar regex para filtrar as linhas onde a coluna 'info' contém a chave 'smile'
        pattern = fr"'{label}'"
        # Filtrar o DataFrame usando str.contains
        filtered_df = VD_LABELED_DT[VD_LABELED_DT['label_measures'].str.contains(pattern, regex=True)]
        filtered_df = filtered_df.reset_index()
        filtered_df['index_diff'] = filtered_df['index'].diff()
        break_points = filtered_df.index[filtered_df['index_diff'] > 1].tolist()
        break_points = [0] + break_points + [len(filtered_df)]

        # Criar uma lista de amostras e adicionar índice numérico para identificação
        filtered_df = filtered_df.drop(columns=['index', 'time_seconds'])
        new_column = pd.Series([np.nan] * len(filtered_df))
        filtered_df.insert(loc=1, column='sample_id', value=new_column)
        #
        
        grouped_samples = []
        for idx in range(len(break_points)-1):
            group = filtered_df.iloc[break_points[idx]:break_points[idx + 1]].copy()
            group['sample_id'] = idx_sample
            for line in group['label_measures']:
                label_dict = eval(line)
                keys = list(label_dict.keys())
                if len(keys) > 1:
                    best_key = keys.pop(0)
                    for key in keys:
                        if label_dict[key][1] < label_dict[best_key][1]:
                            best_key = key
                            break
                else: 
                    best_key = keys[0]
            
            if best_key == label:
                #print('save', best_key, label)
                label_dict = eval(group['label_measures'].iloc[0])
                group['label'] = label
                group['seed_name'] = int(label_dict[label][2])
                grouped_samples.append(group)
                idx_sample+=1
            #else: print('discard', best_key, label)
        
        # Combinar os grupos em um DataFrame final
        converted = pd.concat(grouped_samples).reset_index(drop=True).drop(columns=['index_diff'])
        final_df = pd.concat([final_df, converted]).reset_index(drop=True)
    
    DATASET_SEQUENCES = pd.concat([DATASET_SEQUENCES, final_df]).reset_index(drop=True)

print(f'\nSaving file {OUTPUT_NAME}...')
DATASET_SEQUENCES.to_csv(OUTPUT_NAME)

labels: ['disgust']
..\Dataset\REF-Gold-Label\VD_R_0000000001\VD_LABELED_L0.CSV
labels: ['surprise']
..\Dataset\REF-Gold-Label\VD_R_0000000003\VD_LABELED_L0.CSV
labels: ['anger']
..\Dataset\REF-Gold-Label\VD_R_0000000005\VD_LABELED_L0.CSV
labels: ['happy']
..\Dataset\REF-Gold-Label\VD_R_0000000007\VD_LABELED_L0.CSV
labels: ['surprise']
..\Dataset\REF-Gold-Label\VD_R_0000000008\VD_LABELED_L0.CSV
labels: ['sadness']
..\Dataset\REF-Gold-Label\VD_R_0000000009\VD_LABELED_L0.CSV
labels: ['fear']
..\Dataset\REF-Gold-Label\VD_R_0000000010\VD_LABELED_L0.CSV
labels: ['anger']
..\Dataset\REF-Gold-Label\VD_R_0000000011\VD_LABELED_L0.CSV
labels: ['disgust']
..\Dataset\REF-Gold-Label\VD_R_0000000012\VD_LABELED_L0.CSV
labels: ['happy']
..\Dataset\REF-Gold-Label\VD_R_0000000013\VD_LABELED_L0.CSV
labels: ['surprise']
..\Dataset\REF-Gold-Label\VD_R_0000000014\VD_LABELED_L0.CSV
labels: ['sadness']
..\Dataset\REF-Gold-Label\VD_R_0000000015\VD_LABELED_L0.CSV
labels: ['anger']
..\Dataset\REF-Gold-Label\VD_R